In [1]:
import numpy as np
from netCDF4 import Dataset
import mdtraj

import sys
sys.path.append("../analysis_scripts/")
import misc_tools as misc_tools

# Creating 3D ion density plots around biomolecules

## 1. Center the simulation around the biomolecule of interest 
Simulations are centered around the biomolecule of interest using `VMD`. 
### 1.1 Wrap the solvent around the biomolecule
If the biomolecule is a protein, the following command is entered in the `VMD` `tk console` to center the solvent around biomolecule:
```
package require pbctools
pbc wrap -centersel "protein" -center com -compound residue -all
```
For the DNA dodecamer, the following commands are used:
```
package require pbctools
pbc wrap -centersel "resid 1 to 24" -center com -compound residue -all
```
### 1.2 Align the simulation to the first frame
Using the `RMSD trajectory tool` from `VMD`, the simulations are then aligned to the first PDB file of the first simulation. All the heavy atoms in the macromolecule are used in the alignment. The trajectory is then saved as a `dcd` file.

## 2. Using MDtraj and custom tools to create the density
Using the osmostated DHFR simulations as an example.
### 2.1 Load the simulation data

In [2]:
def get_ion_identities(netcdf_filenames, stride=1):
    """
    Extract the ion densities from a series of netcdf files from a SaltSwap simulation.
    """
    identities = []
    for filename in netcdf_filenames:
        print(filename)
        ncfile = Dataset(filename, 'r')
        data = ncfile.groups['Sample state data']['identities'][:,:]
        identities.append(data[::stride, :])
        ncfile.close()
    identities = np.vstack([*identities])
    return np.array(identities)

# Get the ion indices
file_names = ['out1.nc', 'out2.nc', 'out3.nc']
files = ['../testsystems/dhfr/200mM/' + f for f in file_names]
identities = get_ion_identities(files, stride=1)

# Load the trajectory with the protein centered and aligned.
traj = mdtraj.load('../testsystems/dhfr/200mM/out_all_aligned.dcd', top='../testsystems/dhfr/200mM/out1.pdb')

../testsystems/dhfr/200mM/out1.nc
../testsystems/dhfr/200mM/out2.nc
../testsystems/dhfr/200mM/out3.nc


### 2.2 Extract the coordinates of interest
The identities of the ions are constantly changing in osmostated simulations, so special care must be taken when finding the coordinates. 

In [3]:
# The ions are read by MDtraj to be water molecules.
water_indices = traj.topology.select_atom_indices('water')

# Instead, we need to find which water molecules are in fact ions, and take their coordinates.
[nframes, nwaters] = identities.shape
cation_xyz = []
anion_xyz = []
for frame in range(traj.n_frames - 1):
    # Cations
    indices = [water_indices[water_index] for water_index in range(nwaters) if identities[frame, water_index]==1]
    cation_xyz.append(traj.xyz[frame + 1, indices, :])
    # ("frame + 1" is used instead of just "frame") because the loaded trajectory also contains the initial PDB (out1.pdb)).
    # Anions
    indices = [water_indices[water_index] for water_index in range(nwaters) if identities[frame, water_index]==2]
    anion_xyz.append(traj.xyz[frame + 1, indices, :])

# Simplifying data structure and converting from nanometers to Ångstroms.
cation_xyz = np.vstack([*cation_xyz]) * 10.0   # MDtraj appears to load coordinates in nanometers.
anion_xyz = np.vstack([*anion_xyz]) * 10.0

### 2.3 Get the 3D density
We'll borrow some tools from [ProtoMS 3](http://www.essexgroup.soton.ac.uk/ProtoMS/) to create a smoothed 3D histogram of ion densities. We'll be treating each atom as a hard sphere.

In [4]:
# Some paramters for the spherical smoothing
extent = 2.0   # The effective radius of the atom. The larger this is.
spacing = 1.0  # The grid spacing in Angstroms. 
# The smaller "spacing" is, the finer the 3D mesh, but it will take longer to calculate it.

**Cations**

In [7]:
grid, edges = misc_tools._init_grid(cation_xyz, spacing, 0.0)
for coord in cation_xyz:
    misc_tools._fill_sphere(coord, grid, edges, spacing, extent)
    
grid_cation = grid / np.max(grid) * 100.0   # Setting the isovalues to be between 0 and 100.

# Save the 3D histogram as a dx file.
misc_tools.writeDX(grid_cation, [e[0] for e in edges], spacing, 'dhfr_cation_density_pms.dx')

We can find the total probability a particular isovalue encompasses like so:

In [8]:
isovalue = 15.0

# Normalize the grid:
normed = grid_cation.copy()
normed = normed / np.sum(normed)

print('Proportion of cation density within an isovalue of {0} is {1:3f}%'.format(isovalue, normed[np.where(grid_cation > isovalue)].sum() * 100))

Proportion of cation density within an isovalue of 15.0 is 6.383311%


**Anions**

In [9]:
# Anions
grid, edges = misc_tools._init_grid(anion_xyz, spacing, 0.0)
for coord in anion_xyz:
    misc_tools._fill_sphere(coord, grid, edges, spacing, extent)

grid_anion = grid / np.max(grid) * 100.0   # Setting the isovalues to be between 0 and 100.

# Save the 3D histogram as a dx file.
misc_tools.writeDX(grid_anion, [e[0] for e in edges], spacing, 'dhfr_anion_density_pms.dx')

In [10]:
isovalue = 60.0

# Normalize the grid:
normed = grid_anion.copy()
normed = normed / np.sum(normed)

print('Proportion of anion density within an isovalue of {0} is {1:3f}%'.format(isovalue, normed[np.where(grid_anion > isovalue)].sum() * 100))

Proportion of anion density within an isovalue of 60.0 is 0.857791%
